# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\linoy\Downloads\מטלה 5 למידת מכונה


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

## step 1: simple first try
                    1. lable colume treatment
                    2. vectorization.
                    3. training of a model.
                    4. evaluation of the model with cross validation and average f1.
                    5. prediction on the test set

### 1. lable colume treatment

In [10]:
def gender_to_numeric(df):
    gender_normalized=[]

    for row in df.gender:
        if row=="m":
            gender_normalized.append(0)

        if row=="f":
            gender_normalized.append(1)

    df.gender=gender_normalized
    
    
def numeric_to_gender(df):
    gender_normalized=[]

    for row in df.gender:
        if row==0:
            gender_normalized.append("m")

        if row==1:
            gender_normalized.append("f")

    df.gender=gender_normalized

In [11]:
gender_to_numeric(df_train)
df_train

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",0
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,0
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,1
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",0
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,1
...,...,...
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,0
749,שבוע שעבר העליתי באופן ספונטני רעיון לנסוע עם ...,0
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,0
751,"החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים, ...",1


### 2. vecctorization 

In [12]:
# code section for cheking the vectoraizer

# vec = CountVectorizer(min_df=3)
# X_train_vec = vec.fit_transform(df_train.story)
# df = pd.DataFrame(X_train_vec.toarray(), columns=vec.get_feature_names())
# df
# print("number of hebrew words is: ", len(df.columns))

###  3. training of a model (with vectorization).

In [13]:
def create_simple_pipeline(clf=MultinomialNB(), tfv=False, norm=False, norm_met='l2',ngram_range=(1,1), min_df=3, max_df=1.0, max_features=None):
    if tfv==False:
        if norm== False:
            clf_pipeline= Pipeline([('vect', CountVectorizer(ngram_range=ngram_range, 
                                                             min_df=min_df, max_df= max_df, max_features=max_features)), 
                                    ('clf', clf)])
        else:
              clf_pipeline= Pipeline([('vect', CountVectorizer(ngram_range=ngram_range,
                                                               min_df=min_df, max_df= max_df, max_features=max_features)),
                                    ('norm', preprocessing.Normalizer(norm=norm_met)),
                                    ('clf', clf)])  
    if tfv==True:
        if norm== False:
            clf_pipeline= Pipeline([('vect', TfidfVectorizer(token_pattern=r'[א-ת]+',ngram_range=ngram_range, use_idf=False,sublinear_tf=True,
                                                             min_df=min_df, max_df= max_df, max_features=max_features )), 
                                ('clf', clf)])
        else:
              clf_pipeline= Pipeline([('vect', TfidfVectorizer(token_pattern=r'[א-ת]+',ngram_range=ngram_range,use_idf=False,sublinear_tf=True,
                                                             min_df=min_df, max_df= max_df, max_features=max_features )), 
                                      ('norm', preprocessing.Normalizer(norm=norm_met)),
                                      ('clf', clf)])
            
            
    return clf_pipeline




def fit(clf_pipeline, df_train):
    return clf_pipeline.fit(df_train.story,df_train.gender)

In [14]:
nb_pip= create_simple_pipeline()

pip_trained= fit(nb_pip, df_train)

 ### 4. evaluation of the model with cross validation and average f1.

In [15]:
def f1_avarege_scoring(clf_pipeline):
    scores = cross_val_score(clf_pipeline, df_train['story'], df_train['gender'], 
                             scoring= 'f1_macro',cv=10 )
    print(f"f1_avarege: {scores.mean()}")

In [16]:
f1_avarege_scoring(pip_trained)

f1_avarege: 0.6063709618351841


In [17]:
#reversing the lable colume back to original 
numeric_to_gender(df_train)

### 5. prediction on the test set

In [18]:
def predict(clf_trained_pipeline, x_test):
    
    y_predicted=clf_trained_pipeline.predict(x_test)
    return y_predicted

In [19]:
y_predicted = predict(pip_trained, df_test['story'])

df_test_new= df_test.copy()
df_test_new["gender"]= y_predicted 
numeric_to_gender(df_test_new)

df_test_new.head(-5)

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...,...
313,313,"שלום, לפני כמה חודשים אני ועוד שני חברים מהילד...",m
314,314,"השנה לא הייתה לנו חופשה אמיתית, חווינו סגרים, ...",f
315,315,"לפני כמה חודשים, טסתי לאוקראינה ובאותו ביקור ה...",m
316,316,בנסיעה האחרונה שלי לחול במהלך הקורונה החלטתי ש...,m


## step 2: improvement of the first try
                    1. preprocessing experiments.
                    2. TfidfVectorizer 
                    3. vectorization hyperparameter tuning.
                    4. diffrent normalization of the text.
                    5. additional models.
                    6. model hyperparameter tuning.

### 1. preprocessing experiments. 
            1.punctuation, space and numbers
            2.removel of english words

In [20]:
def base_cleaning(story):
    story = re.sub(r'\d+', '', story)
    story = re.sub(r'[^\w\s]', '', story)
    story = re.sub(r'\s+', ' ', story)
    story = story.strip()
    story = re.sub(r'!', '', story)
    return story

# delete english alphabet
def englishword(story):
    return re.sub('[a-zA-Z]+', '', story)

In [21]:
df_clean= df_train.copy()
df_clean['story'] = df_clean['story'].apply(base_cleaning)
df_clean['story'] = df_clean['story'].apply(englishword)

pip_trained2= fit(nb_pip, df_clean)
f1_avarege_scoring(pip_trained)

f1_avarege: 0.6063709618351841


In [22]:
gender_to_numeric(df_clean)

### 2. TfidfVectorizer 

In [23]:
nb_pip2= create_simple_pipeline(tfv=True)
pip_trained2 = fit(nb_pip2, df_clean)
f1_avarege_scoring(pip_trained2)

f1_avarege: 0.4329778822905296


###  3. vectorization hyperparameter tuning.
            ngram_range ,min_df, max_df, max_features

In [24]:
nb_pip2= create_simple_pipeline(tfv=True,max_df=0.8,ngram_range = (1,7))
pip_trained2 = fit(nb_pip2, df_clean)
f1_avarege_scoring(pip_trained2)

f1_avarege: 0.4329778822905296


In [25]:
nb_pip2= create_simple_pipeline(tfv=True, ngram_range=(1,7), min_df=1, max_df=0.8, max_features=100)
nb_pip2_trained = fit(nb_pip2, df_clean)
f1_avarege_scoring(nb_pip2_trained)

f1_avarege: 0.4329778822905296


In [26]:
nb_pip2= create_simple_pipeline(ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
nb_pip2_trained = fit(nb_pip2, df_clean)
f1_avarege_scoring(nb_pip2_trained)

f1_avarege: 0.6600255792030273


### 4. diffrent normalization of the text.
            1. L1
            2. L2

In [27]:
nb_pip3= create_simple_pipeline(norm=True,ngram_range=(1,2), min_df=0.05, max_df=0.8, max_features=None)
pip_trained3 = fit(nb_pip3, df_clean)
f1_avarege_scoring(pip_trained3)

nb_pip3= create_simple_pipeline(norm=True,  norm_met="l1")
pip_trained3 = fit(nb_pip3, df_clean)
f1_avarege_scoring(pip_trained3)

nb_pip3= create_simple_pipeline(tfv=True, norm=True,  norm_met="l1")
pip_trained3 = fit(nb_pip3, df_clean)
f1_avarege_scoring(pip_trained3)

f1_avarege: 0.4329778822905296
f1_avarege: 0.4329778822905296
f1_avarege: 0.4329778822905296


### 5. additional models.
            1.LinearSVC
            2.KNN
            3.Decision Tree
            4.Perceptron

In [28]:
svc_pip= create_simple_pipeline(clf=LinearSVC(), ngram_range=(1,2), min_df=0.01, max_df=0.8, max_features=None)
pipsvc_trained = fit(svc_pip, df_clean)
print("LinearSVC: ")
f1_avarege_scoring(pipsvc_trained)

knn_pip= create_simple_pipeline(clf=KNeighborsClassifier(n_neighbors=5), ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pipknn_trained = fit(knn_pip, df_clean)
print("KNN: ")
f1_avarege_scoring(pipknn_trained)

dt_pip= create_simple_pipeline(clf=DecisionTreeClassifier(), ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pipdt_trained = fit(dt_pip, df_clean)
print("DecisionTree: ") 
f1_avarege_scoring(pipdt_trained)


pr_pip= create_simple_pipeline(clf=Perceptron(), ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pippr_trained = fit(pr_pip, df_clean)
print("Perceptron: ")
f1_avarege_scoring(pippr_trained)


LinearSVC: 
f1_avarege: 0.6619775182587947
KNN: 
f1_avarege: 0.5335875804995593
DecisionTree: 
f1_avarege: 0.6079655291789313
Perceptron: 
f1_avarege: 0.6606061328906979


### 6. model hyperparameter tuning.
            1. NB: priors, var_smoothing
            2. linearSVC: 
            3. KNN: 
            4. DecisionTree:
            5. Perceptron

In [29]:
vec = CountVectorizer(ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
X_train_vec = vec.fit_transform(df_clean.story)
df = pd.DataFrame(X_train_vec.toarray(), columns=vec.get_feature_names())
df

אבא  אבא שלי  אבי  אביב  אבל  אבל אז  אבל אחרי  אבל אין  אבל אמרתי  \
0      0        0    0     0    4       0         0        0          0   
1      0        0    0     0    1       0         0        0          0   
2      0        0    0     0    5       0         2        0          0   
3      5        5    0     0    2       0         0        0          0   
4      0        0    0     0    2       0         0        0          0   
..   ...      ...  ...   ...  ...     ...       ...      ...        ...   
748    0        0    0     0    0       0         0        0          0   
749    0        0    0     0    3       0         0        0          0   
750    0        0    0     0    2       0         0        0          0   
751    0        0    0     0    0       0         0        0          0   
752    0        0    0     0    2       0         0        0          0   

     אבל אני  אבל בגלל  אבל בכל  אבל גם  אבל הוא  אבל היא  אבל היה  אבל הם  \
0          0         0        0       0        0        0        0       0   
1          1         0        0       0        0        0        0       0   
2          1         0        0       0        0        0        0       0   
3          0         0        0       0        0        0        0       0   
4          0         0        0       0        0        0        0       0   
..       ...       ...      ...     ...      ...      ...      ...     ...   
748        0         0        0       0        0        0        0       0   
749        0         1        0       1        0        0        0       0   
750        0         0        0       0        0        0        0       0   
751        0         0        0       0        0        0        0       0   
752        0         0        0       0        1        0        0       0   

     אבל הפעם  אבל זה  אבל כל  אבל כמובן  אבל לא  אבל עדיין  אבל עם  אבל רק  \
0           0       0       1          0       0          1       0       0   
1           0       0       0          0       0          0       0       0   
2           0       0       0          0       0          0       0       0   
3           1       0       0          0       0          0       0       0   
4           0       0       0          0       0          0       0       0   
..        ...     ...     ...        ...     ...        ...     ...     ...   
748         0       0       0          0       0          0       0       0   
749         0       0       0          0       0          0       0       0   
750         0       1       0          0       1          0       0       0   
751         0       0       0          0       0          0       0       0   
752         0       0       0          0       0          0       0       0   

     אגב  אגם  אדום  אדם  אדמה  אהבה  אהבתי  אהבתי את  אהיה  או  או יותר  \
0      0    1     0    0     0     0      0         0     0   0        0   
1      0    0     0    3     0     0      0         0     0   0        0   
2      0    0     0    0     0     0      0         0     0   0        0   
3      0    0     0    1     0     0      0         0     0   0        0   
4      0    0     0    0     0     0      0         0     0   1        0   
..   ...  ...   ...  ...   ...   ...    ...       ...   ...  ..      ...   
748    0    0     0    1     0     0      0         0     0   1        1   
749    0    0     0    0     0     0      0         0     0   2        0   
750    0    0     0    0     0     0      0         0     0   0        0   
751    0    0     0    0     0     0      0         0     0   1        0   
752    0    0     0    0     0     0      0         0     0   0        0   

     או לא  או משהו  או סתם  אוגוסט  אוהב  אוהבים  אוהבת  אוהדים  אוהל  אוויר  \
0        0        0       0       0     0       0      0       0     0      0   
1        0        0       0       0     1       0      0       0     1      0   
2        0        0       0       0     0       0      0      

### DecisionTree

In [30]:
clf = tree.DecisionTreeClassifier()
grid_param ={
    "min_samples_leaf" : range(1,15,1),
    "max_depth":[5,10,15,30],
    "min_samples_split":range(1,15,1),
}

grid_search = GridSearchCV(estimator=clf , param_grid = grid_param, cv=5, n_jobs=-1, verbose=3, scoring='f1_macro')

In [31]:
grid_search.fit(df,df_clean.gender)

Fitting 5 folds for each of 784 candidates, totalling 3920 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 30],
                         'min_samples_leaf': range(1, 15),
                         'min_samples_split': range(1, 15)},
             scoring='f1_macro', verbose=3)

In [32]:
grid_search.best_params_

{'max_depth': 30, 'min_samples_leaf': 5, 'min_samples_split': 3}

In [33]:
clf=DecisionTreeClassifier(max_depth=30,min_samples_split=10,min_samples_leaf= 1 )
dt_pip2= create_simple_pipeline(clf=clf, ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pipdt_trained2 = fit(dt_pip2, df_clean)
print("DecisionTree: ") 
f1_avarege_scoring(pipdt_trained2)

DecisionTree: 
f1_avarege: 0.6007723116606178


In [34]:
dt_pip2= create_simple_pipeline(clf=clf, tfv=True, ngram_range=(1,7), min_df=1, max_df=0.8, max_features=None)
pipdt_trained2 = fit(dt_pip2, df_clean)
print("DecisionTree: ") 
f1_avarege_scoring(pipdt_trained2)

DecisionTree: 
f1_avarege: 0.5036404487707473


### KNN

In [35]:
def find_best_k_for_KNN(X_train, y_train):
    parameters= {'n_neighbors': [3,5,7,9,11,15,21,23,19,25,27,29]}
    knn= KNeighborsClassifier()
    clf= GridSearchCV(knn,parameters, scoring='f1_macro')
    clf.fit(X_train, y_train)
    best_K= clf.best_params_['n_neighbors']
    best_f1_val= clf.best_score_
    return best_K, best_f1_val

best_K, best_accuracy_score=find_best_k_for_KNN(df,df_clean.gender)
print("bestk", best_K)

bestk 5


the best k=5 is the same value used in the first try therefore we did not try it again.

### LinearSVC

In [36]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100],
              'penalty': ['l1', 'l2'],
              'dual': [False]} 
  
grid = GridSearchCV(estimator=LinearSVC(), param_grid = param_grid, refit = True, verbose = 3, scoring='f1_macro')
  
# fitting the model for grid search
grid.fit(df,df_clean.gender)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ....C=0.01, dual=False, penalty=l1;, score=0.432 total time=   0.1s
[CV 2/5] END ....C=0.01, dual=False, penalty=l1;, score=0.432 total time=   0.0s
[CV 3/5] END ....C=0.01, dual=False, penalty=l1;, score=0.430 total time=   0.0s
[CV 4/5] END ....C=0.01, dual=False, penalty=l1;, score=0.434 total time=   0.0s
[CV 5/5] END ....C=0.01, dual=False, penalty=l1;, score=0.434 total time=   0.0s
[CV 1/5] END ....C=0.01, dual=False, penalty=l2;, score=0.696 total time=   0.0s
[CV 2/5] END ....C=0.01, dual=False, penalty=l2;, score=0.703 total time=   0.0s
[CV 3/5] END ....C=0.01, dual=False, penalty=l2;, score=0.723 total time=   0.0s
[CV 4/5] END ....C=0.01, dual=False, penalty=l2;, score=0.708 total time=   0.0s
[CV 5/5] END ....C=0.01, dual=False, penalty=l2;, score=0.691 total time=   0.1s
[CV 1/5] END .....C=0.1, dual=False, penalty=l1;, score=0.709 total time=   0.1s
[CV 2/5] END .....C=0.1, dual=False, penalty=l1;

GridSearchCV(estimator=LinearSVC(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'dual': [False],
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro', verbose=3)

In [37]:
grid.best_params_

{'C': 0.1, 'dual': False, 'penalty': 'l2'}

In [38]:
clf=LinearSVC(C=0.1, dual=False, penalty='l2')
svc_pip= create_simple_pipeline(clf=clf, ngram_range=(1,2), min_df=0.01, max_df=0.8, max_features=None)
pipsvc_trained = fit(svc_pip, df_clean)
print("LinearSVC with countvectoraizer: ")
f1_avarege_scoring(pipsvc_trained)


svc_pip2= create_simple_pipeline(clf=clf, tfv=True, ngram_range=(1,7), min_df=1, max_df=0.8, max_features=None)
pipsvc_trained2 = fit(svc_pip2, df_clean)
print("LinearSVC with tfidfvectorizer: ")
f1_avarege_scoring(pipsvc_trained2)

LinearSVC with countvectoraizer: 
f1_avarege: 0.6695577799277272
LinearSVC with tfidfvectorizer: 
f1_avarege: 0.4329778822905296


### MultinomialNB

In [39]:
param_grid = {'alpha': [0.0001, 0.05, 0.00001, 1.0],
              'fit_prior': [True,False]} 

  
grid = GridSearchCV(estimator=MultinomialNB(), param_grid = param_grid, refit = True, verbose = 3, scoring='f1_macro')
  
# fitting the model for grid search
grid.fit(df,df_clean.gender)
grid.best_params_

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ......alpha=0.0001, fit_prior=True;, score=0.506 total time=   0.0s
[CV 2/5] END ......alpha=0.0001, fit_prior=True;, score=0.563 total time=   0.0s
[CV 3/5] END ......alpha=0.0001, fit_prior=True;, score=0.488 total time=   0.0s
[CV 4/5] END ......alpha=0.0001, fit_prior=True;, score=0.538 total time=   0.0s
[CV 5/5] END ......alpha=0.0001, fit_prior=True;, score=0.562 total time=   0.0s
[CV 1/5] END .....alpha=0.0001, fit_prior=False;, score=0.506 total time=   0.0s
[CV 2/5] END .....alpha=0.0001, fit_prior=False;, score=0.563 total time=   0.0s
[CV 3/5] END .....alpha=0.0001, fit_prior=False;, score=0.488 total time=   0.0s
[CV 4/5] END .....alpha=0.0001, fit_prior=False;, score=0.538 total time=   0.0s
[CV 5/5] END .....alpha=0.0001, fit_prior=False;, score=0.562 total time=   0.0s
[CV 1/5] END ........alpha=0.05, fit_prior=True;, score=0.600 total time=   0.0s
[CV 2/5] END ........alpha=0.05, fit_prior=True;,

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.0001, 0.05, 1e-05, 1.0],
                         'fit_prior': [True, False]},
             scoring='f1_macro', verbose=3)

{'alpha': 1.0, 'fit_prior': False}

In [40]:
clf=MultinomialNB(alpha=1.0, fit_prior=False)
nb_pip4= create_simple_pipeline(clf=clf, ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pip_trained4 = fit(nb_pip4, df_clean)
print("MultinomialNB: ")
f1_avarege_scoring(pip_trained4 )

nb_pip4= create_simple_pipeline(clf=clf, tfv=True, ngram_range=(1,7), min_df=1, max_df=0.8, max_features=None)
pip_trained4 = fit(nb_pip4, df_clean)
print("MultinomialNB: ")
f1_avarege_scoring(pip_trained4 )

MultinomialNB: 
f1_avarege: 0.6742067635667591
MultinomialNB: 
f1_avarege: 0.4329778822905296


### Perceptron

In [41]:
param_grid = {'alpha': [0.0001, 0.05, 0.00001],
              'penalty': [None, 'l2', 'l1', 'elasticnet']} 

  
grid = GridSearchCV(estimator=Perceptron(), param_grid = param_grid, refit = True, verbose = 3, scoring='f1_macro')
  
# fitting the model for grid search
grid.fit(df,df_clean.gender)
grid.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END ........alpha=0.0001, penalty=None;, score=0.657 total time=   0.1s
[CV 2/5] END ........alpha=0.0001, penalty=None;, score=0.709 total time=   0.1s
[CV 3/5] END ........alpha=0.0001, penalty=None;, score=0.646 total time=   0.1s
[CV 4/5] END ........alpha=0.0001, penalty=None;, score=0.652 total time=   0.1s
[CV 5/5] END ........alpha=0.0001, penalty=None;, score=0.686 total time=   0.1s
[CV 1/5] END ..........alpha=0.0001, penalty=l2;, score=0.660 total time=   0.0s
[CV 2/5] END ..........alpha=0.0001, penalty=l2;, score=0.667 total time=   0.0s
[CV 3/5] END ..........alpha=0.0001, penalty=l2;, score=0.677 total time=   0.1s
[CV 4/5] END ..........alpha=0.0001, penalty=l2;, score=0.625 total time=   0.1s
[CV 5/5] END ..........alpha=0.0001, penalty=l2;, score=0.698 total time=   0.0s
[CV 1/5] END ..........alpha=0.0001, penalty=l1;, score=0.652 total time=   0.2s
[CV 2/5] END ..........alpha=0.0001, penalty=l1;

GridSearchCV(estimator=Perceptron(),
             param_grid={'alpha': [0.0001, 0.05, 1e-05],
                         'penalty': [None, 'l2', 'l1', 'elasticnet']},
             scoring='f1_macro', verbose=3)

{'alpha': 1e-05, 'penalty': 'elasticnet'}

In [42]:
clf=Perceptron(tol=1e-3, random_state=42, alpha=0.00001, max_iter=10, penalty='elasticnet')
pr_pip2= create_simple_pipeline(clf=clf, ngram_range=(1,2), min_df=0.01, max_df=0.9, max_features=None)
pippr_trained2 = fit(pr_pip2, df_clean)
print("Perceptron with countvectoraizer: ") 
f1_avarege_scoring(pippr_trained2)

Perceptron with countvectoraizer: 
f1_avarege: 0.649248144129709


In [43]:
clf=Perceptron(tol=1e-3, random_state=42, alpha=0.00001, max_iter=10)
pr_pip2= create_simple_pipeline(clf=clf, tfv=True, ngram_range=(1,7), min_df=1, max_df=0.8, max_features=None)
pippr_trained2 = fit(pr_pip2, df_clean)
print("Perceptron with tfidfvectorizer: ")
f1_avarege_scoring(pippr_trained2)

Perceptron with tfidfvectorizer: 
f1_avarege: 0.6971751046329588


In [44]:
numeric_to_gender(df_clean)

## final prediction on test set:

In [45]:
y_predicted = predict(pippr_trained2, df_test['story'])

df_test_new= df_test.copy()
df_test_new["gender"]= y_predicted 
numeric_to_gender(df_test_new)

df_test_new.head(-5)

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...,...
313,313,"שלום, לפני כמה חודשים אני ועוד שני חברים מהילד...",m
314,314,"השנה לא הייתה לנו חופשה אמיתית, חווינו סגרים, ...",f
315,315,"לפני כמה חודשים, טסתי לאוקראינה ובאותו ביקור ה...",m
316,316,בנסיעה האחרונה שלי לחול במהלך הקורונה החלטתי ש...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [47]:
df_predicted.to_csv('classification_results.csv',index=False)